In [4]:
#Change to point dataset
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle"""
#import os
#os.environ['DDE_BACKEND'] = 'tensorflow.compat.v1'
import deepxde as dde
import numpy as np
import tensorflow as tf

In [16]:
# Custom data class to handle x, y, A, and B
class CustomData(dde.data.Data):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.num_points = X.shape[0]
        
    def train_next_batch(self, batch_size):
        idx = np.random.choice(self.num_points, batch_size, replace=False)
        return self.X[idx], self.Y[idx]

    def test(self):
        return self.X, self.Y

    def losses(self, targets, outputs, loss_fn):
        return [loss_fn(targets, outputs)]

    def data_id(self):
        return id(self.X)

    def get_data(self):
        return self.X, self.Y

    def transform_data(self, X=None, Y=None, Z=None):
        if X is not None:
            self.X = X
        if Y is not None:
            self.Y = Y
        if Z is not None:
            raise ValueError("Data transformation for Z is not supported.")
        return self

    def num_inputs(self):
        return self.X.shape[1]

    def num_outputs(self):
        return self.Y.shape[1]



In [27]:
# Define the first neural network from (x, y) to A
class FirstNetwork(tf.keras.Model):
    def __init__(self, activation, kernel_initializer):
        super(FirstNetwork, self).__init__()
        self.dense1 = tf.keras.layers.Dense(20, activation=activation, kernel_initializer=kernel_initializer)
        self.dense2 = tf.keras.layers.Dense(20, activation=activation, kernel_initializer=kernel_initializer)
        self.dense3 = tf.keras.layers.Dense(10, kernel_initializer=kernel_initializer)  # A has 10 dimensions

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

# Define the second neural network from A to B
class SecondNetwork(tf.keras.Model):
    def __init__(self, activation, kernel_initializer):
        super(SecondNetwork, self).__init__()
        self.dense1 = tf.keras.layers.Dense(20, activation=activation, kernel_initializer=kernel_initializer)
        self.dense2 = tf.keras.layers.Dense(20, activation=activation, kernel_initializer=kernel_initializer)
        self.dense3 = tf.keras.layers.Dense(1, kernel_initializer=kernel_initializer)  # B has 1 dimension

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

# Define the composite network
class CompositeNetwork(tf.keras.Model):
    def __init__(self, activation, kernel_initializer):
        super(CompositeNetwork, self).__init__()
        self.first_network = FirstNetwork(activation, kernel_initializer)
        self.second_network = SecondNetwork(activation, kernel_initializer)

    def call(self, inputs):
        a = self.first_network(inputs)
        b = self.second_network(a)
        return b


In [28]:
data_points = np.matrix([[0.05, 1e5,    9.86688614e-35],
                        [0.35,  18e4,   2.80E-20],
                        [25.4,  1214e4, 40.4857407],
                        [40.75, 1848e4, 79.3099365],
                        [48.35, 1999e4, 96.3708344],
                        [0.6,   24e4,   1.37249327e-17],
                        [1.55,  169e4,  0.0405000916],
                        [30.15, 267e4,  16.3269901],
                        [43.2,  1541e4, 83.7792282],
                        [36.75, 1704e4, 69.8383331]])

num_samples = 1000
X = np.random.rand(num_samples, 2)  # x, y
A = np.random.rand(num_samples, 10)  # A
B = np.random.rand(num_samples, 1)  # B
# Create the custom data object


In [40]:
# Define the PDE
def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_y = dde.grad.jacobian(y, x, i=0, j=1)
    return dy_x + dy_y - y  # Example PDE: dy/dx + dy/dy - y = 0

# Define the geometry
geom = dde.geometry.Rectangle([-1, -1], [1, 1])

# Define the data object using the PDE
data = dde.data.DataSet(X, B)

# Initialize the composite neural network with activation and kernel_initializer
activation = tf.nn.relu
kernel_initializer = 'glorot_uniform'
composite_nn = CompositeNetwork(activation, kernel_initializer)

# Define a custom neural network wrapper to integrate with DeepXDE
class CustomNN(dde.maps.NN):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def call(self, inputs, training=None):
        return self.model(inputs)

# Wrap the composite network in the custom neural network
net = CustomNN(composite_nn)

# Create a model with the specified network and custom data
model = dde.Model(data, net)

# Compile the model
model.compile("adam", lr=0.001)

# Train the model
losshistory, train_state = model.train(epochs=10000)


AttributeError: 'NoneType' object has no attribute 'astype'

In [13]:

D = 3.8e-1#mm^2/s
l = 1 #mm
t = 1 #s
geom = dde.geometry.Interval(0, l)
timedomain = dde.geometry.TimeDomain(0, t)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

def pde(x, y):
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t- D*dy_xx


T=np.reshape(data_points[:, 1], (-1,1))
X=np.reshape(data_points[:, 0], (-1,1))
C=np.reshape(data_points[:, 2:], (-1,1))

bc = dde.icbc.DirichletBC(geomtime, lambda x: (x[:, 0:1]/ l) * 100, lambda _, on_boundary: on_boundary)
#bc_b = dde.icbc.DirichletBC(geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=1)
ic = dde.icbc.IC(geomtime, lambda x: 0, lambda _, on_initial: on_initial)


observe_x= np.reshape(np.hstack((X,T)),(-1,2))
observe_y = dde.icbc.PointSetBC(observe_x, C, component=0)

data = CustomData(X, B)

layer_size = [2] + [64] * 5 + [1]
activation ="tanh"
initializer = "Glorot normal"
composite_nn = CompositeNetwork()
net = dde.nn.pFNN(composite_nn)
model = dde.Model(data, net)
model.compile("adam", lr=1e-4)
model.train(iterations=20000)
model.compile("L-BFGS")
losshistory, train_state = model.train()
#early_stopping = dde.callbacks.EarlyStopping(min_delta=1e-6, patience=500)
#losshistory, train_state = model.train(iterations=100, callbacks=[early_stopping])
dde.saveplot(losshistory, train_state, issave=True, isplot=True)



TypeError: Can't instantiate abstract class CustomData with abstract methods test, train_next_batch